In [1]:
# predict HK stocks according to US stocks
    # there is jet-lag between US stocks and HK stocks
    # US stock markets have much influence over HK stock markets
    # exploit the information from US stock markets to predict HK stock markets

In [2]:
# base data design: incremental data
    # base data: one year ago - last business day (namely, one year until current business day)
        # base data: training + testing (75% : 25%)
# predict current business day

# 1. it means every day the base-data should be updated, 
        # remove the data of the business day of one year ago
        # add  the data of the latest business day
    # for example, 
        # today is 2022-11-18, 
        # base-data should be (2021-11-18 ~ 2022-11-17) 
        # use the base-data to train & test model, 
        # then the model predict today/current business day: rise or fall

In [2]:
!pip install yfinance
import yfinance as yf # https://pypi.org/project/yfinance/
import math
### the meaning of prediction about stock market
from numpy.core.numeric import ones_like
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 769 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
def get_data_once():
    us_stock_ticker = ["^IXIC", "AAPL", "GOOGL", "AMZN", "MSFT", "BABA", "PDD", "JD", "NTES", "BIDU", "MPNGY", "NIO", "TCOM", "LI", "ZTO", "TME", "XPEV"]
    hk_stock_ticker = ["9988.HK", "3690.HK", "0700.HK", "9618.HK", "0981.HK", "9999.HK", "1810.HK", "1024.HK", "9888.HK", "2015.HK", "9866.HK"]
    ticker_list = us_stock_ticker+hk_stock_ticker
    df_dic = {}
    start_time="2020-01-01"
    end_time="2022-12-31"
    for tn in ticker_list:
        df_data = yf.download(tickers=tn, start=start_time, end=end_time) 
        df_dic[tn] = df_data
    return df_dic

def search_stock(df_dic, tn, st, et):
    tmp_df = df_dic[tn].copy()
    tmp_df['ymd_time'] = tmp_df.index
    tmp_df["ymd_time"] = tmp_df["ymd_time"].astype(str)
    tmp_df["ymd_time"] = tmp_df["ymd_time"].str.slice(0,10)
    tmp_df["date_num"] = tmp_df["ymd_time"].str.slice(0,4) + tmp_df["ymd_time"].str.slice(5,7) + tmp_df["ymd_time"].str.slice(8,10)
    tmp_df["date_num"] = tmp_df["date_num"].astype(int)
    stn = int(st[:4] + st[5:7] + st[8:10])
    etn = int(et[:4] + et[5:7] + et[8:10])
    #print(stn, etn)
    tmp_df = tmp_df[ tmp_df["date_num"]>=stn]
    tmp_df = tmp_df[ tmp_df["date_num"]<etn]
    del tmp_df["ymd_time"]
    del tmp_df["date_num"]
    return tmp_df

df_dic = None
if df_dic is None:
    df_dic = get_data_once()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [71]:
# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2022-10-09"):
  #df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 
  df_data = search_stock(df_dic, ticker_name, start_time, end_time)
  return df_data

# calculate the daily return by (current_index - previous_index) / previous_index
def calculate_daily_return(df_data, OHLC_index="Close"):
  name1 = OHLC_index+"_previous"
  df_data[name1] = df_data[OHLC_index].shift(1)
  name2 = OHLC_index+"_delta"
  df_data[name2] = df_data[OHLC_index] - df_data[name1]
  name3 = OHLC_index+"_return"
  df_data[name3] = df_data[name2] / df_data[name1]
  del df_data[name1]
  del df_data[name2]
  new_feature = name3
  return df_data #, new_feature

# calculate the daily change of points & volumes
    # by (current_index - previous_index) / previous_index
    # by (current_volume - previous_volume) / previous_volume
def calculate_daily_change(df_data):
  # , OHLC_index="Close"
  df_data = calculate_daily_return(df_data, "Close")
  df_data = calculate_daily_return(df_data, "Volume")
  return df_data #, new_feature

"""# convert the time to be string type: yyyy-mm-dd
def get_ymt_date(df_data):
  df_data["ymd_time"] = df_data.index
  df_data["ymd_time"] = df_data["ymd_time"].astype(str)
  df_data["ymd_time"] = df_data["ymd_time"].str.slice(0,10)
  return df_data"""

# convert the time to be string type: yyyy-mm-dd
  # get month number & week number
def get_ymt_date(df_data):
  df_data["day_week"] = df_data.index.dayofweek
  df_data["ymd_time"] = df_data.index
  df_data["ymd_time"] = df_data["ymd_time"].astype(str)
  df_data["ymd_time"] = df_data["ymd_time"].str.slice(0,10)
  # month
  df_data["monthNumber"] = df_data["ymd_time"].str.slice(5,7)
  df_data["monthNumber"] = df_data["monthNumber"].astype(int)
  # week
  df_data['ymd_time'] = pd.to_datetime(df_data['ymd_time'], errors ='coerce')
  df_data['weekNumber'] = df_data['ymd_time'].dt.isocalendar().week
  #
  df_data["ymd_time"] = df_data.index
  df_data["ymd_time"] = df_data["ymd_time"].astype(str)
  df_data["ymd_time"] = df_data["ymd_time"].str.slice(0,10)
  return df_data

# convert dataframe to dict-type data
    # key: date time
    # value: features such as Close_return & Volume_return
def date_features_dict(df_data, features=["ymd_time", "Close_return", "Volume_return"]):
    tmp_df = df_data[features].copy()
    date_list = list(tmp_df["ymd_time"])
    close_return_list = list(tmp_df["Close_return"])
    volume_return_list = list(tmp_df["Volume_return"])
    dic = {}
    ii = 0
    while ii<len(tmp_df):
        d, cr, vr = date_list[ii], close_return_list[ii], volume_return_list[ii]
        dic[d] = [cr, vr]
        ii += 1
    return dic

# get latest US stock data to predict HK stock
def get_multiple_stocks(refer_list, st, et):
    df_reference = pd.DataFrame()
    for tn in refer_list:
        tmp_df = get_df_data(tn, st, et)
        tmp_df = calculate_daily_change(tmp_df)
        tmp_df = get_ymt_date(tmp_df)
        df_reference['ymd_time'] = tmp_df['ymd_time']
        df_reference[tn+"_Close_return"] = tmp_df[ 'Close_return' ]
        df_reference[tn+"_Volume_return"] = tmp_df[ 'Volume_return' ]
    return df_reference

# get label
def date_label(df_target, features):
    tmp_df = df_target[features].copy()
    date_list = list(tmp_df["ymd_time"])
    close_return_list = list(tmp_df["Close_return"])
    dic = {}
    ii = 0
    while ii<len(tmp_df):
        d, cr = date_list[ii], close_return_list[ii]
        if cr>0:
            dic[d] = 1
        else:
            dic[d] = 0
        ii += 1
    return dic

# get features
def date_features(df_reference, refer_list):
    new_refer_list = ["ymd_time"]
    for tn in refer_list:
        new_refer_list.append(tn + "_Close_return")
        new_refer_list.append(tn + "_Volume_return")
    tmp_df = df_reference[new_refer_list].copy()
    dic = {}
    ii = 0
    while ii<len(tmp_df):
        row = tmp_df.iloc[ii]
        ymd_time = row['ymd_time']
        features = []
        for tn in new_refer_list[1:]:
            features.append( row[tn] )
        dic[ymd_time] = features
        ii += 1
    return dic

# get date before today by n days
def previous_date(current, days_num=1):
    current = datetime.datetime.strptime(current, "%Y-%m-%d").date() # %H:%M:%S
    previous = current - datetime.timedelta(days=days_num)
    return str(previous)

# get date after today by n days
def next_date(current, days_num=1):
    current = datetime.datetime.strptime(current, "%Y-%m-%d").date() # %H:%M:%S
    next_date = current + datetime.timedelta(days=days_num)
    return str(next_date)

# search previous business day in US stock market
    # according to business day in HK stock market
def last_US_day(HK_day, us_business_days):
    for days_num in range(1, 30):
        US_day = previous_date(HK_day, days_num)
        if US_day in us_business_days:
            return US_day
    return HK_day

# 

# 
def merge_features_labels(us_dic, hk_dic, refer_list, target, week_info, hk_date_list):
    xy_data = []
    prev_day = 5
    for HK_day, label in hk_dic.items():
        US_day = last_US_day(HK_day, us_business_days)
        if HK_day == US_day:
            continue
        features = us_dic[US_day]
        row = [HK_day, label, US_day] + features
        # week info
        wd = datetime.datetime.strptime(HK_day, "%Y-%m-%d").weekday()
        row = row + week_info[wd]
        # previous trend
        previous_trend = get_previous_business_day(HK_day, hk_date_list, prev_day)
        for pt in previous_trend:
          close = search_movement(target, pt)
          #print(pt, close)
          row.append( close )
        xy_data.append( row )
    # X, y = [], []
    cols = []
    cols.append(target+"_date")
    cols.append("label")
    cols.append("refer_date")
    for tn in refer_list:
        cols.append(tn + "_Close_return")
        cols.append(tn + "_Volume_return")
    cols.append("fall_ratio")
    cols.append("rise_ratio")
    for ind in range(prev_day):
      cols.append("previous"+str(ind+1))
    df = pd.DataFrame(xy_data, columns = cols)
    return df




def scale_list_values(tmp_list):
  tmp = []
  for it in tmp_list:
    tmp.append(it*100)
  return tmp


# get data-sets for training & testing models from dataframe
def dataframe_xy(df_data):
    X, y = [], []
    for i in range(len(df_data)):
        row = list( df_data.iloc[i] )
        [HK_day, label, US_day] = row[:3]
        features = row[3:]
        #
        fg = False
        for v in features:
            if math.isnan(v):
                fg = True
                break
        if fg:
            continue
        #
        X.append( scale_list_values(features) )
        y.append(label)
    return X, y

# sample data for training & testing model
def sample_dataset(df_data, ran_seed, sample_number=-1):
    # balance the positive and negative samples
    positive_df = df_data[ df_data['label']==1 ].copy()
    negative_df = df_data[ df_data['label']==0 ].copy()
    positive_df = positive_df.dropna()
    negative_df = negative_df.dropna()
    #print( "business days: ", len(df_data), "rise vs fall: ", len(positive_df), len(negative_df) )
    if sample_number==-1:
        sample_number = min( len(positive_df), len(negative_df) )
    X_pos_data, y_pos_data = dataframe_xy(positive_df.sample(n=sample_number, random_state=ran_seed))
    X_neg_data, y_neg_data = dataframe_xy(negative_df.sample(n=sample_number, random_state=ran_seed))
    X_data = X_pos_data + X_neg_data
    y_data = y_pos_data + y_neg_data
    #print( "sampled business days: ", len(X_data), "rise vs fall: ", len(X_pos_data), len(X_neg_data) )
    return X_data, y_data

# data-set split and train models
def split_dataset(X_data, y_data, ran_seed):
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=ran_seed, shuffle=True)
    #print( "training data: ", len(X_train), len(y_train) )
    #print( "testing data: ", len(X_test), len(y_test) )
    return X_train, X_test, y_train, y_test

# train LR model
def train_LR_model(X_train, X_test, y_train, y_test):    
    # LR
    LR1 = LogisticRegression()
    LR1.fit(X_train, y_train)
    y_pred = LR1.predict(X_test)
    score = LR1.score(X_test, y_test)
    #print(classification_report(y_test, y_pred))
    #print("LR", score)    
    return LR1, score

# train SVM model
def train_SVM_model(X_train, X_test, y_train, y_test):
    # SVM
    SVM1 = svm.SVC(kernel='linear', probability=True)  # solver='lbfgs', , max_iter=1000 * 1000 * 20
    SVM1.fit(X_train, y_train)
    y_pred = SVM1.predict(X_test)
    score = SVM1.score(X_test, y_test)
    #print(classification_report(y_test, y_pred))
    #print("SVM", score)
    return SVM1, score

# train RF model
def train_RF_model(X_train, X_test, y_train, y_test, ran_seed=0):
    # RF
    RF1=RandomForestClassifier(n_estimators=100, random_state=ran_seed)
    RF1.fit(X_train, y_train)
    y_pred = RF1.predict(X_test)
    score = RF1.score(X_test, y_test)
    #print(classification_report(y_test, y_pred))
    #print("RF", score)
    return RF1, score

def train_RF_model_update(X_train, X_test, y_train, y_test):
  RF1, score = None, 0
  for i in range(10):
    tmp1, tmp2 = train_RF_model(X_train, X_test, y_train, y_test, i)
    if tmp2>score:
      RF1, score = tmp1, tmp2
  return RF1, score


#
def get_features(st, et, refer_list, ind):
    df_reference = get_multiple_stocks(refer_list, st, et)
    features = []
    for tn in refer_list:
        date_time = list(df_reference.index)[ind]
        close_point = list(df_reference[ tn+"_Close_return" ])[ind]
        volume_point = list(df_reference[tn+"_Volume_return" ])[ind]
        print("date_time:", date_time, tn, close_point*100, volume_point*100)
        features.append( close_point )
        features.append( volume_point )
    return features

#
def get_features_update(st, et, target, refer_list, which_date, HK_day, week_info, hk_date_list):
    df_reference = get_multiple_stocks(refer_list, st, et)
    df_reference = df_reference[ df_reference['ymd_time']==which_date ]
    #print(df_reference)
    ind = -1
    features = []
    for tn in refer_list:
        date_time = list(df_reference.index)[ind]
        close_point = list(df_reference[ tn+"_Close_return" ])[ind]
        volume_point = list(df_reference[tn+"_Volume_return" ])[ind]
        print("date_time: ", str(date_time)[:10], tn, close_point*100, volume_point*100)
        features.append( close_point )
        features.append( volume_point )
    #
    wd = datetime.datetime.strptime(HK_day, "%Y-%m-%d").weekday()
    features = features + week_info[wd]
    print("date_time: ", HK_day, "fall_ratio", week_info[wd][0])
    print("date_time: ", HK_day, "rise_ratio", week_info[wd][1])
    #
    prev_day = 5
    previous_trend = get_previous_business_day(HK_day, hk_date_list, prev_day)
    for pt in previous_trend:
      close = search_movement(target, pt)
      print("date_time: ", pt, "trend", close*100)
      features.append( close*100 )
    return features

# 
def search_movement(target, prediction_date_HK):
    dt1, dt2 = previous_date(prediction_date_HK, 10), next_date(prediction_date_HK, 10)
    df_data = search_stock(df_dic, target, dt1, dt2)
    df_data = calculate_daily_change(df_data)
    df_data = get_ymt_date(df_data)
    df_data = df_data[df_data['ymd_time']==prediction_date_HK]
    Close_return = df_data.iloc[-1]['Close_return']
    return Close_return

def weekday_ratio(df_data):
  dic = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0], 4:[0,0], 5:[0,0], 6:[0,0]}
  L = len(df_data)
  i = 0
  while i<L:
    row = df_data.iloc[i]
    Close_return = row['Close_return']
    day_week = row['day_week']
    if Close_return>0:
      dic[day_week][1] += 1
    else:
      dic[day_week][0] += 1
    i += 1
  #
  tmp = {}
  for k, v in dic.items():
    [f, r] = v
    p0 = (f+1)/(f+1+r+1) #round((f+1)/(f+1+r+1)*100, 2)
    p1 = (r+1)/(f+1+r+1) # round((r+1)/(f+1+r+1)*100, 2)
    tmp[k] = [ p0, p1 ]
  return tmp

#
def get_previous_business_day(current, date_list, num):
  tmp_list = []
  day_num = 1
  while len(tmp_list)!=num:
    pv = previous_date(current, day_num)
    if pv in date_list:
      tmp_list.append(pv)
    day_num += 1
  return tmp_list

#
tmp_stock = get_df_data('9988.HK', "2020-01-01", "2022-12-31")
tmp_stock = calculate_daily_change(tmp_stock)
tmp_stock = get_ymt_date(tmp_stock)
hk_date_list = list(tmp_stock['ymd_time'])

tmp_stock = get_df_data('AAPL', "2020-01-01", "2022-12-31")
tmp_stock = calculate_daily_change(tmp_stock)
tmp_stock = get_ymt_date(tmp_stock)
us_date_list = list(tmp_stock['ymd_time'])

hk_date_list, us_date_list
print()

# get_df_data('3690.HK', "2022-01-01", "2022-12-31").tail(10)

In [73]:
to_date = "2022-11-21"
prediction_log = []
for to_date in hk_date_list[-1:]:
    from_date = previous_date(to_date, 365)
    prediction_date_HK = to_date
    
    day_num = 1
    prediction_date_US = previous_date(prediction_date_HK, day_num)
    while prediction_date_US not in us_date_list:
        prediction_date_US = previous_date(prediction_date_HK, day_num)
        day_num += 1
        if day_num>30:
            break

    ### base-data 
    st, et = from_date, to_date

    # target
    target = '9988.HK'
    df_target = get_df_data(target, st, et)
    df_target = calculate_daily_change(df_target)
    df_target = get_ymt_date(df_target)
    features=["ymd_time", "Close_return"] # , "Volume_return"
    hk_dic = date_label(df_target, features)
    hk_business_days = list(hk_dic.keys())
    df_target

    week_info = weekday_ratio(df_target)
    print("week_info: ", week_info)

    # refer list
    refer_list=["^IXIC", "AAPL", "GOOGL", "AMZN", "BABA", "PDD", "JD", "MPNGY", "TME", "BIDU"] # "^IXIC", "^DJI", "^GSPC"
    df_reference = get_multiple_stocks(refer_list, st, et)
    us_dic = date_features(df_reference, refer_list)
    us_business_days = list(us_dic.keys())
    df_reference

    # merge
    df_merge = merge_features_labels(us_dic, hk_dic, refer_list, target, week_info, hk_date_list)
    df_merge

    #
    print( "target: \t HKT", df_target.iloc[0]['ymd_time'], df_target.iloc[-1]['ymd_time'] )
    print( "reference: \t UST", df_reference.iloc[0]['ymd_time'], df_reference.iloc[-1]['ymd_time'] )
    date1, date2 = target+"_date", "refer_date"
    print( "merge: \t HKT - UST", df_merge.iloc[0][date1], df_merge.iloc[0][date2] )
    print( "merge: \t HKT - UST", df_merge.iloc[-1][date1], df_merge.iloc[-1][date2] )
    print( "prediction: \t HKT", prediction_date_HK, "\t according to: \t UST", prediction_date_US )


    ### refine models
    search_parameters = {"LR":[], "SVM":[], "RF":[]}
    sample_number = -1
    ran_seed1, ran_seed2 = 6, 9
    for ran_seed1 in range(0, 10):
        X_data, y_data = sample_dataset(df_merge, ran_seed1, sample_number)
        for ran_seed2 in range(0, 10):
            X_train, X_test, y_train, y_test = split_dataset(X_data, y_data, ran_seed2)
            #
            LR1, score = train_LR_model(X_train, X_test, y_train, y_test)
            search_parameters["LR"].append( (score, ran_seed1, ran_seed2) )
            SVM1, score = train_SVM_model(X_train, X_test, y_train, y_test)
            search_parameters["SVM"].append( (score, ran_seed1, ran_seed2) )
            RF1, score = train_RF_model_update(X_train, X_test, y_train, y_test)
            search_parameters["RF"].append( (score, ran_seed1, ran_seed2) )
        print( ran_seed1, (ran_seed1*10+10), "%" )
    print("LR: \t", sorted(search_parameters["LR"])[-3:])
    print("SVM: \t", sorted(search_parameters["SVM"])[-3:])
    print("RF: \t", sorted(search_parameters["RF"])[-3:])

    # 
    # fit model
    #sample_number = -1
    #ran_seed1, ran_seed2 = 6, 0
    score, ran_seed1, ran_seed2 = sorted(search_parameters["LR"])[-1]
    X_data, y_data = sample_dataset(df_merge, ran_seed1, sample_number)
    X_train, X_test, y_train, y_test = split_dataset(X_data, y_data, ran_seed2)
    LR1, score = train_LR_model(X_train, X_test, y_train, y_test)
    y_pred = LR1.predict(X_test)
    print(classification_report(y_test, y_pred))

    score, ran_seed1, ran_seed2 = sorted(search_parameters["SVM"])[-1]
    X_data, y_data = sample_dataset(df_merge, ran_seed1, sample_number)
    X_train, X_test, y_train, y_test = split_dataset(X_data, y_data, ran_seed2)
    SVM1, score = train_SVM_model(X_train, X_test, y_train, y_test)
    y_pred = SVM1.predict(X_test)
    print(classification_report(y_test, y_pred))

    score, ran_seed1, ran_seed2 = sorted(search_parameters["RF"])[-1]
    X_data, y_data = sample_dataset(df_merge, ran_seed1, sample_number)
    X_train, X_test, y_train, y_test = split_dataset(X_data, y_data, ran_seed2)
    RF1, score = train_RF_model_update(X_train, X_test, y_train, y_test)
    y_pred = RF1.predict(X_test)
    print(classification_report(y_test, y_pred))

    try:
        # predict next day
        predict_features = get_features_update('2020-01-01', '2022-12-31', target, refer_list, prediction_date_US, prediction_date_HK, week_info, hk_date_list)
        
        # 
        rise_or_fall1 = LR1.predict([predict_features])
        prob = LR1.predict_proba([predict_features])
        print( "LR predict today: ", rise_or_fall1, prob )
        rise_or_fall2 = SVM1.predict([predict_features])
        prob = SVM1.predict_proba([predict_features])
        print( "SVM predict today: ", rise_or_fall2, prob )
        rise_or_fall3 = RF1.predict([predict_features])
        prob = RF1.predict_proba([predict_features])
        print( "RF predict today: ", rise_or_fall3, prob )

        # 
        real_signal = "unknown"
        try:
          real_signal = search_movement(target, prediction_date_HK)
        except:
          real_signal = "unknown"
          print( "there is no such real label: for HKT", prediction_date_HK )
        print(rise_or_fall1[0], rise_or_fall2[0], rise_or_fall3[0], " vs ", real_signal)
        prediction_log.append( (rise_or_fall1[0], rise_or_fall2[0], rise_or_fall3[0], real_signal) )
    except:
        print(prediction_date_US, " does not exist in US business day")
    print()

week_info:  {0: [0.7142857142857143, 0.2857142857142857], 1: [0.5490196078431373, 0.45098039215686275], 2: [0.6226415094339622, 0.37735849056603776], 3: [0.6296296296296297, 0.37037037037037035], 4: [0.5294117647058824, 0.47058823529411764], 5: [0.5, 0.5], 6: [0.5, 0.5]}
target: 	 HKT 2021-11-18 2022-11-17
reference: 	 UST 2021-11-18 2022-11-17
merge: 	 HKT - UST 2021-11-19 2021-11-18
merge: 	 HKT - UST 2022-11-17 2022-11-16
prediction: 	 HKT 2022-11-18 	 according to: 	 UST 2022-11-17
0 10 %
1 20 %
2 30 %
3 40 %
4 50 %
5 60 %
6 70 %
7 80 %
8 90 %
9 100 %
LR: 	 [(0.8125, 0, 7), (0.8333333333333334, 1, 7), (0.8541666666666666, 8, 5)]
SVM: 	 [(0.7916666666666666, 8, 5), (0.7916666666666666, 9, 3), (0.8125, 1, 7)]
RF: 	 [(0.7916666666666666, 7, 7), (0.8125, 6, 7), (0.8125, 8, 5)]
              precision    recall  f1-score   support

           0       0.80      0.91      0.85        22
           1       0.91      0.81      0.86        26

    accuracy                           0.85     

In [74]:
prediction_log
counter = 0
for it in prediction_log:
    p1, p2, p3, r = it
    if r>0:
      r = 1
    else:
      r = 0
    print(p1, p2, p3, " vs ", r)

0 0 0  vs  1
